# Emoji–Hill Cipher Examples
This notebook reproduces the encryption and decryption examples from the paper.

In [ ]:
import numpy as np
import emoji
import pandas as pd

# Hybrid alphabet (N=40) consistent with the paper
alphabet = [
    "A","B","C","D","E","F","G","H","I","J",
    "K","L","M","N","O","P","Q","R","S","T",
    "U","V","W","X","Y","Z"," ", "!", "0","1",
    "2","3","4",
    emoji.emojize(":brain:"),                   
    emoji.emojize(":rocket:"),                  
    emoji.emojize(":bar_chart:"),               
    emoji.emojize(":robot:"),                   
    emoji.emojize(":lock:"),                    
    emoji.emojize(":smiling_face_with_smiling_eyes:"), 
    emoji.emojize(":puzzle_piece:")             # PAD
]

N = len(alphabet)
phi = {ch: i for i, ch in enumerate(alphabet)}
phi_inv = {i: ch for i, ch in enumerate(alphabet)}

print("Alphabet size:", N)
print("Sample indices:", {ch: phi[ch] for ch in [emoji.emojize(':brain:'), 'A', 'X']})

In [ ]:
# Key matrix (same as in the paper)
K = np.array([[1, 2, 3],
              [0, 1, 4],
              [5, 6, 0]])
det = int(round(np.linalg.det(K)))
print("det(K) =", det)

# Inverse over Z_40
K_inv = np.linalg.inv(K).astype(float)
K_adj = np.round(det * K_inv).astype(int)  # adjugate
K_inv_mod = (K_adj % N)  # since det=1
print("K^-1 mod 40:\n", K_inv_mod)

In [ ]:
def encode_message(msg):
    return [phi[ch] for ch in msg]

def decode_message(indices):
    return ''.join(phi_inv[i] for i in indices)

def blockify(seq, r=3):
    # Padding with PAD if needed
    while len(seq) % r != 0:
        seq.append(phi[emoji.emojize(":puzzle_piece:")])
    return [seq[i:i+r] for i in range(0, len(seq), r)]

def encrypt_block(block):
    x = np.array(block)
    y = K.dot(x) % N
    return y.tolist()

def decrypt_block(block):
    y = np.array(block)
    x = K_inv_mod.dot(y) % N
    return x.astype(int).tolist()

In [ ]:
# Example 1: Encryption
plaintext = [emoji.emojize(":brain:")] + list("DATA SCIENCE IS FUN!") \
             + [emoji.emojize(":rocket:"), emoji.emojize(":bar_chart:")]

# Encode and blockify
x_indices = encode_message(plaintext)
blocks = blockify(x_indices, r=3)

rows = []
for j, b in enumerate(blocks, 1):
    y_raw = K.dot(b)
    y_mod = (y_raw % N).tolist()
    decoded = [phi_inv[i] for i in y_mod]
    rows.append([j, b, y_raw.tolist(), y_mod, decoded])

import pandas as pd
df_enc = pd.DataFrame(rows, columns=["j","x (indices)","Kx (pre-mod)","Kx mod 40","Decoded"])
df_enc

In [ ]:
# Ciphertext final
ciphertext = ''.join(''.join(row[-1]) for row in rows)
print("Ciphertext:\n", ciphertext)

In [ ]:
# Example 2: Decryption
cipher_example = "UMMA" + emoji.emojize(":puzzle_piece:") + "MT0JTE" \
    + emoji.emojize(":puzzle_piece:") + "!FDX2VPE" \
    + emoji.emojize(":smiling_face_with_smiling_eyes:") + "YG3" \
    + "DH" + emoji.emojize(":rocket:") \
    + emoji.emojize(":smiling_face_with_smiling_eyes:") + emoji.emojize(":lock:") + "0" \
    + "CC0YM4AUNSL4SSN"

cipher_indices = encode_message(list(cipher_example))
blocks_c = blockify(cipher_indices, r=3)

rows_dec = []
for j, b in enumerate(blocks_c, 1):
    x_raw = K_inv_mod.dot(b)
    x_mod = (x_raw % N).astype(int).tolist()
    decoded = [phi_inv[i] for i in x_mod]
    rows_dec.append([j, b, x_raw.tolist(), x_mod, decoded])

df_dec = pd.DataFrame(rows_dec, columns=["j","y (indices)","K^-1 y (pre-mod)","K^-1 y mod 40","Decoded"])
df_dec

In [ ]:
# Plaintext final recovered
plaintext_recovered = ''.join(''.join(row[-1]) for row in rows_dec)
print("Recovered plaintext:\n", plaintext_recovered)